# Estimate Tests

- Create the internal consistency code
- build the diff file code for estimates 

In [37]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np

What internal consistency will do 


Functions: 
1. Function where you pass in the three levels and it checks for equivalence between them and outputs differences, maybe a helpful dataframe 
2. Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 
- Could output a dataframe where all the sums are there then filter for greater than zero, or output the whole dataframe 
3. Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 
4. Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 
5. Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [2]:
def internal_consistency():
    '''
    Along with an individual check for each column sum 

    Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 

Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 
    
    '''

# Internal Consistency

In [38]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

data = {}
for path in files:
    data[path.split('\\')[1][:-4]] = pd.read_csv(path)

In [39]:
data.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

### Compare Totals 
This is comparing the totals accross CPA, Jur, and Region to see if they are equivalent. 

In [96]:
def compare_totals(cpa_df, jur_df, region_df):
    """
    Aggregates sum of values by year in cpa, jurisdiction, and region level and compares them together to see if they match. CPA is removed as many areas calculated in the other georgraphy 
    """
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df
    
    non_matches_dict = {}
    jur_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(jur_totals[col], cpa_totals[col]).all():
            jur_non_matches.append(col)
                        
    if len(jur_non_matches) == len(cpa_totals.columns):
        non_matches_dict['jurisdiction'] = 'all columns did not match.'
    elif len(jur_non_matches) == 0:
        non_matches_dict['jurisdiction'] = 'all columns matched.'
    else:
        if len(jur_non_matches) > 10:
            non_matches_dict['jurisdiction'] = f"{len(jur_non_matches)} columns did not match out of {len(cpa_totals.columns)} columns."
        else:
            non_matches_dict['jurisdiction'] = ', '.join(jur_non_matches) + ' columns did not match.'
    
    reg_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(reg_totals[col], cpa_totals[col]).all():
            reg_non_matches.append(col)
    if len(reg_non_matches) == len(cpa_totals.columns):
        non_matches_dict['region'] = 'all columns did not match.'
    elif len(reg_non_matches) == 0:
        non_matches_dict['region'] = 'all columns matched.'
    else:
        if len(reg_non_matches) > 10:
            non_matches_dict['region'] = f"{len(reg_non_matches)} columns did not match out of {len(cpa_totals.columns)}."
        else:
            non_matches_dict['region'] = ', '.join(reg_non_matches) + ' columns did not match.'
    
    return non_matches_dict

In [97]:
compare_totals(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'jurisdiction': 'all columns matched.', 'region': 'all columns matched.'}

### Ethnicity population check 
Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

In [76]:
def ethnicity_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Hispanic'] + df['Non-Hispanic, White'] + df['Non-Hispanic, Black'] + df['Non-Hispanic, American Indian or Alaska Native'] + df['Non-Hispanic, Asian'] + df['Non-Hispanic, Hawaiian or Pacific Islander'] + df['Non-Hispanic, Other'] + df['Non-Hispanic, Two or More Races']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results


In [79]:
ethnicity_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Age Group Comparison Check
Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

In [91]:
def age_group_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Under 5'] + df['5 to 9'] + df['10 to 14'] + df['15 to 17'] + df['18 and 19'] + df['20 to 24'] + df['25 to 29'] + df['30 to 34'] + df['35 to 39'] + df['40 to 44'] + df['45 to 49'] + df['50 to 54'] + df['55 to 59'] + df['60 and 61'] + df['62 to 64'] + df['65 to 69'] + df['70 to 74'] + df['75 to 79'] + df['80 to 84'] + df['85 and Older']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [92]:
age_group_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Gender Population test 
Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [94]:
def gender_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Female'] + df['Male']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [95]:
gender_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

# Creating Diff Files

In [109]:
def diff_file(df1, df2, geo_level=None):
    '''
    df1 minus df2
    
    '''
    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id']) - df2.set_index(['cpa', 'yr_id'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id']) - df2.set_index(['jurisdiction', 'yr_id'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id']) - df2.set_index(['region', 'yr_id'])).reset_index()

    
    return new_df

In [110]:
diff_file(data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_jurisdiction_QA'], geo_level='jurisdiction')

,jurisdiction,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,Mobile Home,Single-family Detached,Single-family Attached,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other,Female,Male
0,Carlsbad,2010,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Carlsbad,2011,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Carlsbad,2012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlsbad,2013,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Carlsbad,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Vista,2016,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,Vista,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,Vista,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,Vista,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
